In [1]:
import dostools
import importlib
import numpy as np
import pickle
import torch
import sys
import matplotlib.pyplot as plt
import copy
from tqdm import tqdm
import matplotlib
import time
torch.set_default_dtype(torch.float64) 


In [2]:
import dostools.datasets.data as data
import dostools.utils.utils as utils

n_structures = 1039
np.random.seed(0)
n_train = int(0.8 * n_structures)
train_index = np.arange(n_structures)
np.random.shuffle(train_index)
test_index = train_index[n_train:]
train_index = train_index[:n_train]

with torch.no_grad():
    structures = data.load_structures(":")
    n_structures = len(structures) #total number of structures
    for structure in structures:#implement periodicity
        structure.wrap(eps = 1e-12) 
    n_atoms = np.zeros(n_structures, dtype = int) #stores number of atoms in each structures
    for i in range(n_structures):
        n_atoms[i] = len(structures[i])

    #eigenergies, emin, emax = dostools.src.datasets.data.load_eigenenergies(unpack = True, n_structures = len(structures))
    xdos = torch.tensor(data.load_xdos())
    ldos = torch.tensor(data.load_ldos())
    ldos *= 2

    print ("ldos shape is {}".format(ldos.shape))
    mean_dos_per_atom = ldos[train_index].mean(axis = 0) #only calculated for train set to prevent data leakage
    print ("mean dos shape is {}".format(mean_dos_per_atom.shape))
    
    
    y_pw = ldos - mean_dos_per_atom
    y_lcdf = torch.cumsum(y_pw, dim = 1)
    _, pc_vectors = utils.build_pc(ldos[train_index], mean_dos_per_atom[None,:], n_pc = 10)
    y_pc = utils.build_coeffs(ldos - mean_dos_per_atom[None,:], pc_vectors)
    Silicon = data.load_features()
    kMM = data.load_kMM()

ldos shape is torch.Size([1039, 778])
mean dos shape is torch.Size([778])
Variance covered with 10 PCs is = 0.9871211778950163


In [3]:
#Testing linalg solver
Features = torch.rand(1000,1000)
Target = torch.sum(Features, dim = 1)

In [8]:
HYPER_PARAMETERS = {
    "cutoff": 4.0,#6.0,#4.0,
    "max_radial": 8,#12,#8,
    "max_angular": 6,#9,#6,
    "atomic_gaussian_width": 0.45,
    "center_atom_weight": 1.0,
    "radial_basis":{
        "Gto":{}
    },
    "cutoff_function":{
        "Step":{}, #maybe 
    },
    "radial_scaling":{
        "Willatt2018":{
        'exponent': 5,
        'rate' : 1,
        'scale' : 3.,
        },
    },
}


In [27]:
from dostools.datasets import dataset
importlib.reload(dataset)
Silicon = dataset.TensorFeatures(structures, HYPERS = HYPER_PARAMETERS)

  0%|          | 0/999 [00:00<?, ?it/s]

In [33]:
import rascaline
calculator = rascaline.SoapPowerSpectrum(**HYPER_PARAMETERS)
descriptors = calculator.compute(structures)
descriptors.keys_to_samples("species_center")
descriptors.keys_to_properties(["species_neighbor_1", "species_neighbor_2"])

TensorMap with 1 blocks
keys: ['species_center']
             14

In [115]:
from skcosmo.feature_selection import FPS

n_refs_list = [10,100,200,300,500,1000]
features = []
references_list = []
for i in n_refs_list:
    n_refs = i
    n_atoms = descriptors.block(0).values.shape[0]
    n_structures = np.unique(descriptors.block(0).samples["structure"])
    feature = torch.zeros(len(n_structures), n_refs)
    atom_descriptors = torch.tensor(descriptors.block(0).values)
    atom_descriptors = torch.nn.functional.normalize(atom_descriptors, dim = 1)
    selector = FPS(n_to_select = n_refs,
               progress_bar = True,
               score_threshold = 1e-12,
               full = False,
               initialize = 0
              )
    selector.fit(atom_descriptors.T)
    references = selector.transform(atom_descriptors.T).T
    references_list.append(references)
    atomkernel_descriptors = torch.pow(atom_descriptors @ references.T, 2)
    for structure_i in n_structures:
        a_i = descriptors.block(0).samples["structure"] == structure_i
        feature[structure_i, :] = torch.sum(atomkernel_descriptors[a_i, :], axis = 0)/np.sum(a_i)
    
    features.append(feature)
    

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/199 [00:00<?, ?it/s]

  0%|          | 0/299 [00:00<?, ?it/s]

  0%|          | 0/499 [00:00<?, ?it/s]

  0%|          | 0/999 [00:00<?, ?it/s]

In [123]:
ref = references_list[1]
(ref @ ref.T)[0]

array([1.        , 0.96632736, 0.86986185, 0.90142101, 0.85702559,
       0.83123826, 0.9725669 , 0.92358503, 0.93671668, 0.95975516,
       0.94431939, 0.89330837, 0.94240001, 0.87765371, 0.96293411,
       0.93096273, 0.95057861, 0.92019552, 0.94001133, 0.77642409,
       0.88282718, 0.83647533, 0.89917572, 0.80321235, 0.8373274 ,
       0.93604405, 0.90523543, 0.85469427, 0.93380464, 0.98492286,
       0.95946996, 0.97065462, 0.95290117, 0.81265865, 0.86292156,
       0.92098442, 0.86749019, 0.95964987, 0.9106295 , 0.98141447,
       0.95330467, 0.94920076, 0.97550968, 0.86911248, 0.90232848,
       0.86298411, 0.88082986, 0.83393233, 0.85578193, 0.86891578,
       0.8256393 , 0.80365978, 0.88065644, 0.88192544, 0.92652282,
       0.92661697, 0.89107062, 0.92364297, 0.81822085, 0.9332443 ,
       0.85046409, 0.85899991, 0.88690372, 0.88675728, 0.89323711,
       0.93011608, 0.89591242, 0.94752131, 0.96382072, 0.92933705,
       0.95189218, 0.89926599, 0.93237377, 0.93900668, 0.92292

In [103]:
selector.X_selected_

array([[ 0.4923334 ,  0.34286131,  0.64359844, ...,  0.54424215,
         0.62024215,  0.48815145],
       [ 0.27081983,  0.1828978 ,  0.35657218, ...,  0.29623356,
         0.33448261,  0.26189865],
       [ 0.0359634 ,  0.0309307 ,  0.19616021, ...,  0.04133072,
         0.05713942,  0.0451317 ],
       ...,
       [-0.01586898,  0.00155525,  0.00267389, ...,  0.00543415,
         0.00345414,  0.00836395],
       [ 0.01439919,  0.00298427,  0.00480714, ...,  0.00482142,
         0.00328138,  0.00672281],
       [ 0.01541978,  0.00175395,  0.0026493 , ...,  0.00228569,
         0.00178636,  0.00292208]])

In [114]:
references.shape

(50, 448)

In [113]:
np.sum((references @ references.T)[0]>0.9)

30

In [96]:
references @ references.T

array([[1.        , 0.96632736, 0.86986185, ..., 0.92992223, 0.96870089,
        0.97600572],
       [0.96632736, 1.        , 0.90978503, ..., 0.89939367, 0.94623085,
        0.94465566],
       [0.86986185, 0.90978503, 1.        , ..., 0.92596412, 0.90371158,
        0.90835236],
       ...,
       [0.92992223, 0.89939367, 0.92596412, ..., 1.        , 0.9401854 ,
        0.97571001],
       [0.96870089, 0.94623085, 0.90371158, ..., 0.9401854 , 1.        ,
        0.97080159],
       [0.97600572, 0.94465566, 0.90835236, ..., 0.97571001, 0.97080159,
        1.        ]])

In [104]:
references

array([[ 4.92333403e-01,  2.70819830e-01,  3.59634029e-02, ...,
        -1.58689810e-02,  1.43991933e-02,  1.54197787e-02],
       [ 4.11635953e-01,  2.21560647e-01,  5.10777589e-02, ...,
         3.25896568e-04,  2.17442138e-02,  1.39618361e-02],
       [ 3.40482618e-01,  1.81236431e-01,  4.79457651e-02, ...,
         8.77447087e-03,  7.48807291e-03,  3.40624759e-03],
       ...,
       [ 4.31384174e-01,  2.38385080e-01,  2.68996786e-02, ...,
         6.15506436e-04,  1.05952854e-03,  8.97425091e-04],
       [ 5.04393928e-01,  2.74378861e-01,  4.10343368e-02, ...,
         1.80856769e-03,  7.18183711e-03,  4.73787403e-03],
       [ 4.47899226e-01,  2.48465055e-01,  2.75294591e-02, ...,
         8.42089482e-04,  6.16194472e-03,  5.28876391e-03]])

In [89]:
torch.linalg.cond(features[2])

tensor(3.6432e+08)

In [43]:
features[0]

tensor([[1.0000, 0.6910, 0.8774,  ..., 0.8087, 0.8686, 0.6336],
        [0.9954, 0.6765, 0.8903,  ..., 0.8480, 0.8828, 0.6544],
        [0.9974, 0.6843, 0.8924,  ..., 0.8378, 0.8890, 0.6556],
        ...,
        [0.9579, 0.7939, 0.8952,  ..., 0.8066, 0.8550, 0.6117],
        [0.9567, 0.7964, 0.8965,  ..., 0.8048, 0.8549, 0.6107],
        [0.9563, 0.8012, 0.8983,  ..., 0.8070, 0.8558, 0.6106]])

In [125]:
main = features[2]

Features = main[train_index]#Silicon.Features['structure_avekerneldescriptors'][train_index]
test_features = main[test_index]#Silicon.Features['structure_avekerneldescriptors'][test_index]
Target = ldos[train_index]
test_target = ldos[test_index]
m = Features.shape[1]
regularization = 1e-3
reg = regularization * torch.eye(m)
reg[-1, -1] = 0
A = torch.vstack([Features, reg])
b = torch.vstack([Target, torch.zeros(m,Target.shape[1])])
weights = torch.linalg.lstsq(A, b, rcond=1e-10).solution
pred = Features @ weights 
test_pred = test_features @ weights

loss_dos = loss.t_get_rmse(pred, Target, xdos, perc = True)
test_loss_dos = loss.t_get_rmse(test_pred, test_target, xdos, perc = True)
print ("Regularization: {}".format(regularization))
print ("The train error is {:.4} for n_refs = {}".format(loss_dos,m))
print ("The test error is {:.4} for n_refs = {}".format(test_loss_dos, m))

Regularization: 0.001
The train error is 11.36 for n_refs = 200
The test error is 12.98 for n_refs = 200


In [130]:
kMM

0.9999999999999994

In [164]:
kMM = references_list[i] @ references_list[i].T

In [167]:
np.sum(kMM<0)

0

In [200]:
import scipy 

i = 2
main = features[i]
kMM = references_list[i] @ references_list[i].T

Features = main[train_index]#Silicon.Features['structure_avekerneldescriptors'][train_index]
test_features = main[test_index]#Silicon.Features['structure_avekerneldescriptors'][test_index]
Target = ldos[train_index]
test_target = ldos[test_index]
m = Features.shape[1]
regularization = 0
rtkMM = scipy.linalg.sqrtm(kMM)
reg = torch.tensor(regularization * rtkMM)
reg[-1, -1] = 0
A = torch.vstack([Features, reg])
b = torch.vstack([Target, torch.zeros(m,Target.shape[1])])



weights = torch.linalg.lstsq(A, b, driver = "gelsd", rcond = 1e-10).solution
pred = Features @ weights 
test_pred = test_features @ weights

loss_dos = loss.t_get_rmse(pred, Target, xdos, perc = True)
test_loss_dos = loss.t_get_rmse(test_pred, test_target, xdos, perc = True)
print ("Regularization: {}".format(regularization))
print ("The train error is {:.4} for n_refs = {}".format(loss_dos,m))
print ("The test error is {:.4} for n_refs = {}".format(test_loss_dos, m))

Regularization: 0
The train error is 5.284 for n_refs = 200
The test error is 23.26 for n_refs = 200


In [ ]:
#Lets go for 200

In [201]:
torch.linalg.cond(A)

tensor(4.0676e+08)

In [299]:
z = features[2]
y = references_list[2]
m = z.shape[1]
regularization = 1
kMM = y @ y.T
rtkMM = scipy.linalg.sqrtm(kMM)
reg = torch.tensor(regularization * rtkMM)
reg[-1, -1] = 0
m_train_loss = 0
m_test_loss = 0

A = torch.vstack([z[train_index], reg])
b = torch.vstack([ldos[train_index], torch.zeros(m,ldos.shape[1])])


weights = torch.linalg.lstsq(A, b, driver = "gelsd", rcond = 1e-10).solution

train_pred = z[train_index] @ weights
test_pred = z[test_index] @ weights

train_loss = loss.t_get_rmse(train_pred, ldos[train_index], xdos, perc = True)
test_loss = loss.t_get_rmse(test_pred, ldos[test_index], xdos, perc = True)

print (train_loss, test_loss)

torch.Size([1031, 200])
torch.Size([1031, 778])
tensor(11.2006) tensor(12.6701)


In [352]:

z = torch.hstack([features[2], torch.ones(features[2].shape[0]).view(-1,1)]).double()
m = z.shape[1]
kMM = references_list[2] @ references_list[2].T
rtkMM = scipy.linalg.sqrtm(kMM)
reg = torch.hstack([torch.real(torch.tensor(regularization * rtkMM)), torch.zeros(kMM.shape[0]).view(-1,1)])
reg = torch.vstack([reg, torch.zeros(m)])

#     reg[-1, -1] = 0
m_train_loss = 0
m_test_loss = 0

A = torch.vstack([z[train_index], reg])
b = torch.vstack([ldos[train_index], torch.zeros(m,ldos.shape[1])])
weights = torch.linalg.lstsq(A, b, driver = "gelsd", rcond = 1e-10).solution


train_pred = z[train_index] @ weights

test_pred = z[test_index] @ weights

train_loss = loss.t_get_rmse(train_pred, ldos[train_index], xdos, perc = True)
test_loss = loss.t_get_rmse(test_pred, ldos[test_index], xdos, perc = True)

print (train_loss, test_loss)

tensor(10.9057) tensor(12.4247)


In [353]:
weights = torch.linalg.lstsq(A, B, driver = "gelsd", rcond = 1e-10).solution


train_pred = z[train_index] @ weights

test_pred = z[test_index] @ weights

train_loss = loss.t_get_rmse(train_pred, ldos[train_index], xdos, perc = True)
test_loss = loss.t_get_rmse(test_pred, ldos[test_index], xdos, perc = True)

print (train_loss, test_loss)

tensor(10.9057) tensor(12.4247)


In [357]:
train_loss2

tensor(10.9057)

In [337]:
def test(A,b):
    weights = torch.linalg.lstsq(A, b, driver = "gelsd", rcond = 1e-10).solution

    train_pred = z[train_index] @ weights

    test_pred = z[test_index] @ weights

    train_loss = loss.t_get_rmse(train_pred, ldos[train_index], xdos, perc = True)
    test_loss = loss.t_get_rmse(test_pred, ldos[test_index], xdos, perc = True)

    print (train_loss, test_loss)

In [319]:
test(A,b)

tensor(10.9057) tensor(12.4247)


In [322]:
test(C,d)

tensor(10.9057) tensor(12.4247)


In [252]:
kMM = references_list[3] @ references_list[3].T
rtkMM = scipy.linalg.sqrtm(kMM)
reg = torch.tensor(regularization * rtkMM)

In [333]:
train_pred = z[train_index] @ weights

test_pred = z[test_index] @ weights

train_loss = loss.t_get_rmse(train_pred, ldos[train_index], xdos, perc = True)
test_loss = loss.t_get_rmse(test_pred, ldos[test_index], xdos, perc = True)

print (train_loss, test_loss)

tensor(10.9057) tensor(12.4247)


In [341]:
test(A,b)

RuntimeError: torch.linalg.lstsq: input.size(-2) should match other.size(-2)

In [339]:
A.shape

torch.Size([831, 778])

In [342]:
b.shape

torch.Size([208, 778])

In [363]:
find_reg_CV(features[2], ldos, xdos, references_list[2], 1, train_index, test_index)

(tensor(10.1991), tensor(16.3043), tensor(10.9057), tensor(12.4247))

In [379]:
#find reg using CV and see what happens
from sklearn.model_selection import KFold
n_refs_list = [10,100,200,300,500,1000]
regs = [0, 1e-10, 1e-5, 1e-3, 1e-1, 1, 10, 100, 1000]

def find_reg_CV(features, target, xdos, references, regularization, train_index, test_index, cv = 2):
    kf = KFold(n_splits = cv)
    features = torch.hstack([features, torch.ones(features.shape[0]).view(-1,1)]).type(dtype = torch.complex128)
    m = features.shape[1]
    kMM = references @ references.T
    rtkMM = scipy.linalg.sqrtm(kMM)
    reg = torch.hstack([(torch.tensor(regularization * rtkMM)), torch.zeros(kMM.shape[0]).view(-1,1)])
    reg = torch.vstack([reg, torch.zeros(m)]).type(dtype = torch.complex128)
    target = target.type(dtype = torch.complex128)
#     reg[-1, -1] = 0
    m_train_loss = 0
    m_test_loss = 0
    
    
    
    for (cv_train_index, cv_test_index) in kf.split(features[train_index]):
        A = torch.vstack([features[train_index[cv_train_index]], reg])
        b = torch.vstack([target[train_index[cv_train_index]], torch.zeros(m,target.shape[1])])

        weights = torch.linalg.lstsq(A, b, driver = "gelsd", rcond = 1e-10).solution
        
        train_pred = features[train_index[cv_train_index]] @ weights
        test_pred = features[train_index[cv_test_index]] @ weights
        
        train_loss = torch.abs(loss.t_get_rmse(train_pred, target[train_index[cv_train_index]], xdos, perc = True))
        test_loss = torch.abs(loss.t_get_rmse(test_pred, target[train_index[cv_test_index]], xdos, perc = True))
    
        
        m_train_loss += train_loss/cv
        m_test_loss += test_loss/cv
        
    
    A = torch.vstack([features[train_index], reg])
    b = torch.vstack([target[train_index], torch.zeros(m,target.shape[1])])
    
    weights = torch.linalg.lstsq(A, b, driver = "gelsd", rcond = 1e-10).solution
    
    
    
    train_pred = features[train_index] @ weights
    
    test_pred = features[test_index] @ weights

    train_loss = torch.abs(loss.t_get_rmse(train_pred, target[train_index], xdos, perc = True))
    test_loss = torch.abs(loss.t_get_rmse(test_pred, target[test_index], xdos, perc = True))
    
    return m_train_loss, m_test_loss, train_loss, test_loss



In [380]:

for i, feat in enumerate(features):
    for reg in regs:
        m_trainloss, m_testloss, train_loss, test_loss = find_reg_CV(feat, ldos, xdos, references_list[i], reg, train_index, test_index)
        
        print ("The cv_train, cv_test loss at reg: {} is {:.4}, {:.4} for n_ref = {}".format(reg, m_trainloss, m_testloss, n_refs_list[i]))
        print ("The train, test loss at reg: {} is {:.4}, {:.4} for n_ref = {}".format(reg, train_loss, test_loss, n_refs_list[i]))
    print ("---------------------------------------------")

The cv_train, cv_test loss at reg: 0 is 32.26, 33.19 for n_ref = 10
The train, test loss at reg: 0 is 32.5, 32.28 for n_ref = 10
The cv_train, cv_test loss at reg: 1e-10 is 32.26, 33.19 for n_ref = 10
The train, test loss at reg: 1e-10 is 32.5, 32.28 for n_ref = 10
The cv_train, cv_test loss at reg: 1e-05 is 32.26, 33.19 for n_ref = 10
The train, test loss at reg: 1e-05 is 32.5, 32.28 for n_ref = 10
The cv_train, cv_test loss at reg: 0.001 is 32.26, 33.19 for n_ref = 10
The train, test loss at reg: 0.001 is 32.5, 32.28 for n_ref = 10
The cv_train, cv_test loss at reg: 0.1 is 33.97, 34.47 for n_ref = 10
The train, test loss at reg: 0.1 is 33.41, 33.44 for n_ref = 10
The cv_train, cv_test loss at reg: 1 is 41.8, 42.02 for n_ref = 10
The train, test loss at reg: 1 is 39.71, 39.42 for n_ref = 10
The cv_train, cv_test loss at reg: 10 is 84.11, 84.27 for n_ref = 10
The train, test loss at reg: 10 is 75.98, 76.65 for n_ref = 10
The cv_train, cv_test loss at reg: 100 is 99.75, 99.84 for n_ref 

In [371]:
#find reg using CV and see what happens
from sklearn.model_selection import KFold
n_refs_list = [10,100,200,300,500,1000]
regs = [0, 1e-10, 1e-5, 1e-3, 1e-1, 1, 10,100,1000]

def find_reg_CV(features, target, xdos, references, regularization, train_index, test_index, cv = 2):
    kf = KFold(n_splits = cv)
    features = torch.hstack([features, torch.ones(features.shape[0]).view(-1,1)])
    m = features.shape[1]
    reg = torch.tensor(regularization * torch.eye(m))

    reg[-1, -1] = 0
    m_train_loss = 0
    m_test_loss = 0
    
    
    
    for (cv_train_index, cv_test_index) in kf.split(features[train_index]):
        A = torch.vstack([features[train_index[cv_train_index]], reg])
        b = torch.vstack([target[train_index[cv_train_index]], torch.zeros(m,target.shape[1])])

        weights = torch.linalg.lstsq(A, b, rcond = 1e-10).solution
        
        train_pred = features[train_index[cv_train_index]] @ weights
        test_pred = features[train_index[cv_test_index]] @ weights
        
        train_loss = loss.t_get_rmse(train_pred, target[train_index[cv_train_index]], xdos, perc = True)
        test_loss = loss.t_get_rmse(test_pred, target[train_index[cv_test_index]], xdos, perc = True)
    
        
        m_train_loss += train_loss/cv
        m_test_loss += test_loss/cv
        
    
    A = torch.vstack([features[train_index], reg])
    b = torch.vstack([target[train_index], torch.zeros(m,target.shape[1])])
    
    
    weights = torch.linalg.lstsq(A, b, rcond = 1e-10).solution
    
    
    
    train_pred = features[train_index] @ weights
    
    test_pred = features[test_index] @ weights

    train_loss = loss.t_get_rmse(train_pred, target[train_index], xdos, perc = True)
    test_loss = loss.t_get_rmse(test_pred, target[test_index], xdos, perc = True)
    
    return m_train_loss, m_test_loss, train_loss, test_loss




In [ ]:
for i, feat in enumerate(features):
    for reg in regs:
        m_trainloss, m_testloss, train_loss, test_loss = find_reg_CV(feat, ldos, xdos, references_list[i], reg, train_index, test_index)
        
        print ("The cv_train, cv_test loss at reg: {} is {:.4}, {:.4} for n_ref = {}".format(reg, m_trainloss, m_testloss, n_refs_list[i]))
        print ("The train, test loss at reg: {} is {:.4}, {:.4} for n_ref = {}".format(reg, train_loss, test_loss, n_refs_list[i]))
    print ("---------------------------------------------")

reg: 1 for 200 refs kMM
The cv_train, cv_test loss at reg: 1 is 10.2, 16.3 for n_ref = 200
The train, test loss at reg: 1 is 10.91, 12.42 for n_ref = 200

reg: 0.001 for 200 refs L2 
The cv_train, cv_test loss at reg: 0.001 is 10.96, 16.36 for n_ref = 200
The train, test loss at reg: 0.001 is 11.27, 13.24 for n_ref = 20

In [31]:
8.5 - 43.01

-34.51

In [ ]:
2.956 - 37.61

In [21]:
weights.shape

torch.Size([100, 778])

In [127]:
regularization = 100
n_col = Features.shape[1]
reg = regularization * torch.eye(n_col)
reg[n_col-1, n_col-1] = 0
reg_features = Features.T @ Features + reg


# matinv = torch.linalg.inv(Features.T @ Features)
# weight = matinv @ Features.T @ Target

weight2 = torch.linalg.lstsq(((Features.T @ Features)+reg),Features.T @ Target, driver = "gelsd", rcond = 1e-15).solution
preds1 = Features @ weight2
# preds2 = Features @ weight

print ("error1 : {:.4}".format(torch.mean(Target - preds1)**2))
# print ("error2 : {:.4}".format(torch.mean(Target - preds2)**2))

error1 : 1.261e-24


In [126]:
weight2

tensor([1.1406e-01, 9.8752e-02, 7.7868e-02,  ..., 1.1185e-01, 8.9080e-02,
        8.1009e+02])

In [113]:
weight

tensor([152238.9036, -81631.2050,  93461.0305,  ..., 135140.0196,
        -71803.1538, -79685.7966])

In [92]:
torch.mean(Target - preds2)**2

tensor(32.8394)

In [93]:
shifted_weights

tensor([731.6263])